In [1]:
import sys
sys.path.append('..')
from swdf.utils import *
convert_uuids_to_indices()
import sklearn
from tsai.basics import *
my_setup(sklearn)
from matplotlib import dates as mdates
import wandb
from fastai.callback.wandb import WandbCallback
from fastai.callback.progress import ShowGraphCallback

os              : Linux-6.8.0-39-generic-x86_64-with-glibc2.31
python          : 3.10.12
tsai            : 0.3.10
fastai          : 2.7.15
fastcore        : 1.5.49
sklearn         : 1.5.1
torch           : 2.2.2+cu121
device          : 1 gpu (['NVIDIA GeForce RTX 3070 Ti Laptop GPU'])
cpu cores       : 14
threads per cpu : 1
RAM             : 15.28 GB
GPU memory      : [8.0] GB


In [2]:
# This is only needed for the MIT supercloud, to fix fastai's LRFinder error 
if torch.cuda.is_available() and torch.cuda.device_count() == 0:
    from fastai.callback.schedule import LRFinder

    @patch_to(LRFinder)
    def after_fit(self):
        self.learn.opt.zero_grad() # Needed before detaching the optimizer for future fits
        tmp_f = self.path/self.model_dir/self.tmp_p/'_tmp.pth'
        if tmp_f.exists():
            self.learn.load(f'{self.tmp_p}/_tmp', with_opt=True, device='cpu')
            self.tmp_d.cleanup()

In [3]:
config_base = yaml2dict('./config/base.yaml', attrdict=True)
config_dstap = yaml2dict('./config/geodstap.yaml', attrdict=True)
config_train = config_dstap.train
config_data = config_dstap.data

# Merge the two configs (the second one overrides the first one for any keys that are present in both)
config = AttrDict({**config_base,
                   **config_train, 
                   "data":AttrDict({**config_data})})

# Add the architecture config
if config.arch_name.lower() == 'patchtst':
    config.arch = yaml2dict('./config/patchtst.yaml', attrdict=True)
else:
    config.arch = AttrDict()

In [4]:
run = wandb.init(project=config.wandb.project, 
                 config=config,
                 group=config.wandb.group,
                 mode=config.wandb.mode, 
                 anonymous='never') if config.wandb.enabled else None
config = dict2attrdict(run.config) if config.wandb.enabled else config

In [5]:
config

```json
{ 'add_time_channels': False,
  'arch': {'attn_dropout': 0.4, 'd_model': 1024, 'dropout': 0.2, 'n_heads': 32},
  'arch_config_file': 'config/patchtst.yaml',
  'arch_name': 'PatchTST',
  'bs': 64,
  'data': { 'ap_config': './config/geoap.yaml',
            'data_nb': './geodstap_data.ipynb',
            'df_save_path': './preprocessed_data/dataframes/geodstap.pkl',
            'dst_config': './config/geodst.yaml',
            'preproc_pipe_save_path': './preprocessed_data/pipelines/preproc_geodstap.pkl'},
  'deltaHL': 4.0,
  'horizon': 48,
  'init_weights': False,
  'is_optuna_study': False,
  'lookback': 144,
  'lr_max': None,
  'n_epoch': 50,
  'partial_n': 0.001,
  'seed': 42,
  'test_end_datetime': '2018-12-31 12:00:00',
  'test_start_datetime': '2012-10-01 12:00:00',
  'valid_start_datetime': '2018-01-01 12:00:00',
  'wandb': { 'enabled': False,
             'group': None,
             'log_learner': True,
             'mode': 'offline',
             'project': 'swdf'}}
```

In [6]:
df, preproc_pipe = generate_preprocessed_data(config.data)

df.head()

,Datetime,DOY,DST,AP
0,1957-01-01 00:00:00,1,12.0,2
1,1957-01-01 03:00:00,1,9.3,3
2,1957-01-01 06:00:00,1,5.0,3
3,1957-01-01 09:00:00,1,-5.0,22
4,1957-01-01 12:00:00,1,0.0,15


# Train

In [7]:
# In the paper by Licata et al. (2020) (https://agupubs.onlinelibrary.wiley.com/doi/pdf/10.1029/2020SW002496),
# authors use a period from October 2012 through the end of 2018 for the benchmarking.
# Therefore, we will set the test set as the same period for our analysis, 
# using the column Date as the timestamp, from October 2012 to the end of 2018. 
# Everything before the test set will be used for training, and everything after the test set
# will be used for validation
test_start_datetime = config.test_start_datetime
test_end_datetime = config.test_end_datetime
valid_start_datetime = config.valid_start_datetime

In [8]:
data_columns = ['AP', 'DST']
data_columns_time = ['DOY']
data_columns_fcst = data_columns + data_columns_time if config.add_time_channels else data_columns
data_columns

['AP', 'DST']

In [ ]:
# hide

fig, ax = plt.subplots(2, 1, figsize=(20, 5))
plt.subplots_adjust(hspace=0.6)

for i, value in enumerate(data_columns): 
    ax[i].plot(df.Datetime, df[value])
    ax[i].axvspan(df.Datetime.min(), test_start_datetime, alpha=0.5, color='green')
    ax[i].axvspan(test_start_datetime, test_end_datetime, alpha=0.5, color='red')
    ax[i].axvspan(test_end_datetime, df.Datetime.max(), alpha=0.5, color='blue')
    ax[i].set_title(f'{value} - Train (green), Test (red), Validation (blue)')
    ax[i].set_xlabel('Time')
    ax[i].set_ylabel('AP (2nT)')
    ax[i].xaxis.set_major_locator(mdates.YearLocator(2))
    ax[i].xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

    ax[i].xaxis.set_major_locator(mdates.YearLocator(2))
    ax[i].xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    plt.setp(ax[i].get_xticklabels(), rotation=45, ha="right")
plt.show()

In [10]:
splits_ = get_forecasting_splits(df[df.Datetime < valid_start_datetime], 
                             fcst_history=config.lookback, 
                             fcst_horizon=config.horizon, 
                             use_index=False, 
                             test_cutoff_datetime=test_start_datetime, 
                             show_plot=False, 
                             datetime_col='Datetime')
foo = df[df.Datetime >= valid_start_datetime]
bar = get_forecasting_splits(foo, config.lookback, config.horizon, valid_size=0.0, 
                             test_size=0.0, show_plot=False)
val_idxs = L(foo.index[bar[0]].tolist())

splits = (splits_[0], val_idxs, splits_[1])
splits

((#162710) [0,1,2,3,4,5,6,7,8,9...],
 (#19088) [178244,178245,178246,178247,178248,178249,178250,178251,178252,178253...],
 (#15296) [162757,162758,162759,162760,162761,162762,162763,162764,162765,162766...])

In [11]:
train_split = splits[0]
exp_pipe = sklearn.pipeline.Pipeline([
    ('scaler', TSStandardScaler(columns=data_columns)),
], verbose=True)
save_object(exp_pipe, 'tmp/exp_pipe.pkl')
exp_pipe = load_object('tmp/exp_pipe.pkl')

df_scaled = exp_pipe.fit_transform(df.copy(), scaler__idxs = train_split)

df_scaled.head()

tmp directory already exists.
Pipeline saved as tmp/exp_pipe.pkl
[Pipeline] ............ (step 1 of 1) Processing scaler, total=   0.0s


,Datetime,DOY,DST,AP
0,1957-01-01 00:00:00,1,1.156901,-0.584428
1,1957-01-01 03:00:00,1,1.043766,-0.534888
2,1957-01-01 06:00:00,1,0.863589,-0.534888
3,1957-01-01 09:00:00,1,0.444572,0.406378
4,1957-01-01 12:00:00,1,0.654080,0.059596


In [12]:
# We'll approach the time series forecasting task as a supervised learning problem. 
# Remember that tsai requires that both inputs and outputs have the following shape:
# (samples, features, steps)

# To get those inputs and outputs we're going to use a function called 
# `prepare_forecasting_data`` that applies a sliding window along the dataframe
X, y = prepare_forecasting_data(df, fcst_history=config.lookback, 
                                fcst_horizon=config.horizon,
                                x_vars=data_columns, 
                                y_vars=data_columns_fcst)

X.shape, y.shape

((197332, 2, 144), (197332, 2, 48))

In [13]:
thresholds = {
    "AP": [
        (-np.inf, 10),  # Low
        (10, 50),    # Moderate
        (50, np.inf)   # Active
    ],
    "DST": [
        (-30, np.inf),    # G0
        (-50, -30),     # G1
        (-90, -50),     # G2
        (-130, -90),    # G3
        (-350, -130),   # G4
        (-np.inf, -350)    # G5
    ]
}

activity_levels = {
    'AP': ["Low", "Moderate", "Active"],
    'DST': ["G0", "G1", "G2", "G3", "G4", "G5"]
}

In [14]:
df_cat = get_classified_columns(df, thresholds, activity_levels)

def get_weights(column:pd.Series):
    distribution = column.value_counts(normalize=True).to_dict()
    inverted_weights = {k: 1/v for k, v in distribution.items()}
    total_weight = sum(inverted_weights.values())
    weights = [v/total_weight for v in inverted_weights.values()]
    return weights

weights = {column: get_weights(df_cat[df.Datetime < test_start_datetime][f'{column}_Cat']) for column in data_columns}
weights

{'AP': [0.05027553210414406, 0.08743208708643083, 0.862292380809425],
 'DST': [8.803351903177007e-05,
  0.0006020729557296344,
  0.0014018781299297467,
  0.007761028027872513,
  0.01549121753377997,
  0.9746557698336564]}

In [15]:
save_object(weights, 'tmp/weights_geodstap.pkl')
save_object(thresholds, 'tmp/thresholds_geodstap.pkl')

tmp directory already exists.
dict saved as tmp/weights_geodstap.pkl
tmp directory already exists.
dict saved as tmp/thresholds_geodstap.pkl


In [ ]:
from swdf.losses import LossFactory
from swdf.metrics import ValidationMetricsHandler

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
loss = LossFactory(thresholds=thresholds, weights=weights).create(config.loss_func, **config.loss_config).to(device)

metrics_handler = ValidationMetricsHandler(config.metrics)
metrics = metrics_handler.get_metrics()

wandb_callback = WandbCallback(log_preds=False)
cbs = L(wandb_callback) if config.wandb.enabled else L()
learn = TSForecaster(X, y, splits=splits, batch_size=config.bs,
                     pipelines=[preproc_pipe, exp_pipe], 
                     arch=config.arch_name, 
                     metrics=metrics,
                     loss_func=loss,
                     arch_config=dict(config.arch), 
                     init=config.init_weights,
                     cbs= cbs + ShowGraphCallback(), 
                     partial_n=config.partial_n)
learn.to(device)

try:
    lr_max = learn.lr_find().valley if config.lr_max is None else config.lr_max
except:
    lr_max = 1e-3

learn.fit_one_cycle(n_epoch=config.n_epoch, lr_max=config.lr_max)


In [ ]:
# Print the validation loss and save it in case other notebooks (optuna) wants to
# use it for hyperparameter optimization
valid_metrics = learn.recorder.metrics

print(valid_metrics)
%store valid_metrics

1.8741647005081177
Stored 'valid_loss' (float)


In [ ]:
# Save everything
learn.dls.loaders += [learn.dls.valid.new_dl(X[splits[2]], y[splits[2]])] # Add test datalaoder

# Remove the wandb callback to avoid errors when downloading the learner
if config.wandb.enabled:
    learn.remove_cb(wandb_callback)

In [ ]:
best_valid_metrics = None
%store -r best_valid_metrics

# save_all_or_best = (config.is_optuna_study and valid_loss < best_valid_loss) or not config.is_optuna_study
save_all_or_best = (config.is_optuna_study and 
                    metrics_handler.are_best_values(config.main_metric, best_valid_metrics, valid_metrics)) or not config.is_optuna_study


if save_all_or_best:
    best_valid_metrics = valid_metrics
    %store best_valid_metrics                                           

    # Save locally and in wandb if online and enabled
    learn.save_all(path='tmp', verbose=True) 
    if run is not None and config.wandb_mode and config.wandb_log_learner:
        # Save the learner (all tmp/dls, tmp/model.pth, and tmp/learner.pkl). 
        run.log_artifact('tmp', type='learner', name='solfsmy')
    
    # Save the best model
    model = {}
    model['models'] = [learn.model]
    model['lookback'] = config.lookback
    model['horizon'] = config.horizon
    model['data_columns'] = data_columns
    model['thresholds'] = thresholds

    torch.save(model, 'models/best/best_model_solfsmy.pth')

Stored 'best_valid_loss' (float)
Learner saved:
path          = 'tmp'
dls_fname     = '['dls_0.pth', 'dls_1.pth', 'dls_2.pth', 'dls_3.pth']'
model_fname   = 'model_dstap.pth'
learner_fname = 'learner_dstap.pkl'


In [ ]:
# Log the test loss to wandb
test_loss = learn.validate(ds_idx=2)[0]
print(test_loss)
if run is not None:
    run.log(dict(test_loss=test_loss))

2.188929319381714


In [ ]:
if run is not None:
    run.finish()